In [1]:
import numpy as np
from scipy.stats import norm, qmc
import time


def timed(func):
    """Decorator to time a function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper


def black_scholes_call(S0, K, T, r, sigma):
    """Analytical Black-Scholes formula for European call option."""
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


@timed
def european_call_option_mc_sobol(S0, K, T, r, sigma, n_paths, track_convergence=False):
    """
    Prices a European call option using Monte Carlo with Sobol sequences.
    """
    if n_paths <= 0 or not isinstance(n_paths, int):
        raise ValueError("Number of paths must be a positive integer.")
    if T <= 0:
        raise ValueError("Time to maturity must be positive.")

    sampler = qmc.Sobol(d=1, scramble=True)
    sobol_seq = sampler.random(n=n_paths).flatten()
    z = norm.ppf(sobol_seq)

    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    payoff = np.maximum(ST - K, 0)
    discounted_payoff = np.exp(-r * T) * payoff

    option_price = np.mean(discounted_payoff)

    if track_convergence:
        estimates = np.cumsum(discounted_payoff) / np.arange(1, n_paths + 1)
        return option_price, estimates

    return option_price


# === ⚙️ Option Parameters ===
S0 = 100
K = 100
T = 1.0
r = 0.05
sigma = 0.2

# === 🔢 Simulation Control ===
n_paths = 2**21

# === 🧠 Analytical Price ===
bs_price = black_scholes_call(S0, K, T, r, sigma)

# === 🚀 Sobol Monte Carlo Estimate ===
(mc_price, duration) = european_call_option_mc_sobol(S0, K, T, r, sigma, n_paths)

# === 📊 Comparison ===
abs_error = abs(mc_price - bs_price)
rel_error = abs_error / bs_price * 100

print("\n--- European Call Option Pricing Comparison ---")
print(f"Black-Scholes (Analytical):     {bs_price:.8f}")
print(f"Sobol Monte Carlo Estimate:     {mc_price:.8f}")
print(f"Absolute Error:                 {abs_error:.8e}")
print(f"Relative Error:                 {rel_error:.8e}%")
print(f"Monte Carlo Time Elapsed:       {duration:.6f} seconds")



--- European Call Option Pricing Comparison ---
Black-Scholes (Analytical):     10.45058357
Sobol Monte Carlo Estimate:     10.45058159
Absolute Error:                 1.98520454e-06
Relative Error:                 1.89961118e-05%
Monte Carlo Time Elapsed:       0.218843 seconds


In [2]:
import numpy as np
from scipy.stats import norm, qmc
import time


def timed(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper


def black_scholes_call(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


@timed
def european_call_mc_sobol_vr(S0, K, T, r, sigma, n_paths, antithetic=True, control_variate=True):
    if n_paths <= 0 or not isinstance(n_paths, int):
        raise ValueError("n_paths must be a positive integer")
    if T <= 0:
        raise ValueError("Time to maturity must be positive")

    n_paths = n_paths if not antithetic else n_paths // 2

    sampler = qmc.Sobol(d=1, scramble=True)
    z = norm.ppf(sampler.random(n=n_paths).flatten())

    if antithetic:
        z = np.concatenate([z, -z])

    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    discount_factor = np.exp(-r * T)
    call_payoff = np.maximum(ST - K, 0)
    discounted_call = discount_factor * call_payoff

    if control_variate:
        control_payoff = ST
        control_expected = S0 * np.exp(r * T)
        cov = np.cov(discounted_call, control_payoff)[0, 1]
        var_control = np.var(control_payoff, ddof=1)
        beta = cov / var_control
        adjusted_payoff = discounted_call - beta * (control_payoff - control_expected)
        option_price = np.mean(adjusted_payoff)
    else:
        option_price = np.mean(discounted_call)

    return option_price


# Parameters
S0 = 100
K = 100
T = 1.0
r = 0.05
sigma = 0.2
n_paths = 2**25

# Analytical
bs_price = black_scholes_call(S0, K, T, r, sigma)

# Monte Carlo
(mc_price, duration) = european_call_mc_sobol_vr(
    S0, K, T, r, sigma, n_paths,
    antithetic=True,
    control_variate=True
)

# Results
abs_error = abs(mc_price - bs_price)
rel_error = abs_error / bs_price * 100
total_paths = n_paths * 2  # because of antithetic

print("\n--- European Call Option Pricing with Variance Reduction ---")
print(f"Black-Scholes (Analytical):     {bs_price:.8f}")
print(f"MC Estimate (Sobol + VR):       {mc_price:.8f}")
print(f"Absolute Error:                 {abs_error:.8e}")
print(f"Relative Error:                 {rel_error:.8e}%")
print(f"Paths Used:                     {total_paths:,}")
print(f"Time Elapsed:                   {duration:.4f} seconds")



--- European Call Option Pricing with Variance Reduction ---
Black-Scholes (Analytical):     10.45058357
MC Estimate (Sobol + VR):       10.45058353
Absolute Error:                 4.57574334e-08
Relative Error:                 4.37845725e-07%
Paths Used:                     67,108,864
Time Elapsed:                   3.9624 seconds


In [3]:
import numpy as np
from scipy.stats import norm, qmc
import time


def timed(func):
    """A decorator that times the execution of a function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper


def black_scholes_call(S0, K, T, r, sigma):
    """Calculates the price of a European call option using the Black-Scholes formula.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock price.

    Returns:
        float: The Black-Scholes price of the call option.
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


@timed
def european_call_mc_sobol_vr(S0, K, T, r, sigma, n_paths, antithetic=True, control_variate=True):
    """Estimates the price of a European call option using Monte Carlo simulation with Sobol sequence and variance reduction techniques.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock price.
        n_paths (int): Number of Sobol paths to generate (before considering antithetic variates).
        antithetic (bool): Whether to use antithetic variates to reduce variance.
        control_variate (bool): Whether to use the stock price at maturity as a control variate.

    Returns:
        tuple: A tuple containing the estimated option price and the execution duration.
    """
    if not isinstance(n_paths, int) or n_paths <= 0:
        raise ValueError("n_paths must be a positive integer")
    if T <= 0:
        raise ValueError("Time to maturity must be positive")

    # Generate Sobol sequence
    sampler = qmc.Sobol(d=1, scramble=True)
    sobol_samples = sampler.random(n=n_paths)
    z = norm.ppf(sobol_samples.flatten())

    if antithetic:
        z = np.concatenate([z, -z])
        n_total_paths = 2 * n_paths
    else:
        n_total_paths = n_paths

    # Simulate stock prices at maturity
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    # Calculate discounted call payoffs
    discount_factor = np.exp(-r * T)
    call_payoff = np.maximum(ST - K, 0)
    discounted_call = discount_factor * call_payoff

    if control_variate:
        # Control variate: Stock price at maturity
        control_payoff = ST
        control_expected = S0 * np.exp(r * T)

        # Calculate covariance and variance
        covariance = np.cov(discounted_call, control_payoff)[0, 1]
        variance_control = np.var(control_payoff, ddof=1)  # Use sample variance

        # Calculate the optimal beta
        beta = covariance / variance_control

        # Adjust the call price using the control variate
        adjusted_payoff = discounted_call - beta * (control_payoff - control_expected)
        option_price = np.mean(adjusted_payoff)
    else:
        option_price = np.mean(discounted_call)

    return option_price


if __name__ == "__main__":
    # Parameters
    S0 = 100
    K = 100
    T = 1.0
    r = 0.05
    sigma = 0.2
    n_paths = 2**25  # Reduced for faster execution in testing

    # Analytical Black-Scholes price
    bs_price = black_scholes_call(S0, K, T, r, sigma)

    # Monte Carlo simulation
    (mc_price, duration) = european_call_mc_sobol_vr(
        S0, K, T, r, sigma, n_paths,
        antithetic=True,
        control_variate=True
    )

    # Results
    abs_error = abs(mc_price - bs_price)
    rel_error = abs_error / bs_price * 100
    total_paths = n_paths * 2  # because of antithetic

    print("\n--- European Call Option Pricing with Variance Reduction ---")
    print(f"Black-Scholes (Analytical):    {bs_price:.8f}")
    print(f"MC Estimate (Sobol + VR):      {mc_price:.8f}")
    print(f"Absolute Error:                {abs_error:.8e}")
    print(f"Relative Error:                {rel_error:.8e}%")
    print(f"Paths Used:                    {total_paths:,}")
    print(f"Time Elapsed:                  {duration:.6f} seconds")


--- European Call Option Pricing with Variance Reduction ---
Black-Scholes (Analytical):    10.45058357
MC Estimate (Sobol + VR):      10.45058344
Absolute Error:                1.31634472e-07
Relative Error:                1.25958968e-06%
Paths Used:                    67,108,864
Time Elapsed:                  6.606538 seconds


In [4]:
import numpy as np
from scipy.stats import norm, qmc
import time


def timed(func):
    """A decorator that times the execution of a function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper


def black_scholes_call(S0, K, T, r, sigma):
    """Calculates the price of a European call option using the Black-Scholes formula.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock price.

    Returns:
        float: The Black-Scholes price of the call option.
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


@timed
def european_call_mc_sobol_vr(S0, K, T, r, sigma, n_paths, antithetic=True, control_variate=True, stratified=False):
    """Estimates the price of a European call option using Monte Carlo simulation with Sobol sequence and variance reduction techniques.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock price.
        n_paths (int): Number of Sobol paths to generate (before considering antithetic variates).
        antithetic (bool): Whether to use antithetic variates to reduce variance.
        control_variate (bool): Whether to use the stock price at maturity as a control variate.
        stratified (bool): Whether to use stratified sampling on the Sobol sequence.

    Returns:
        tuple: A tuple containing the estimated option price and the execution duration.
    """
    if not isinstance(n_paths, int) or n_paths <= 0:
        raise ValueError("n_paths must be a positive integer")
    if T <= 0:
        raise ValueError("Time to maturity must be positive")

    # Generate Sobol sequence
    sampler = qmc.Sobol(d=1, scramble=True)
    sobol_samples = sampler.random(n=n_paths)

    if stratified:
        # Apply stratified sampling
        stratified_samples = (np.arange(n_paths).reshape(-1, 1) + sobol_samples) / n_paths
        z = norm.ppf(stratified_samples.flatten())
    else:
        z = norm.ppf(sobol_samples.flatten())

    if antithetic:
        z = np.concatenate([z, -z])
        n_total_paths = 2 * n_paths
    else:
        n_total_paths = n_paths

    # Simulate stock prices at maturity
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    # Calculate discounted call payoffs
    discount_factor = np.exp(-r * T)
    call_payoff = np.maximum(ST - K, 0)
    discounted_call = discount_factor * call_payoff

    if control_variate:
        # Control variate: Stock price at maturity
        control_payoff = ST
        control_expected = S0 * np.exp(r * T)

        # Calculate covariance and variance
        covariance = np.cov(discounted_call, control_payoff)[0, 1]
        variance_control = np.var(control_payoff, ddof=1)  # Use sample variance

        # Calculate the optimal beta
        beta = covariance / variance_control

        # Adjust the call price using the control variate
        adjusted_payoff = discounted_call - beta * (control_payoff - control_expected)
        option_price = np.mean(adjusted_payoff)
    else:
        option_price = np.mean(discounted_call)

    return option_price


if __name__ == "__main__":
    # Parameters
    S0 = 100
    K = 100
    T = 1.0
    r = 0.05
    sigma = 0.2
    n_paths = 2**27 # Reduced for faster execution in testing

    # Analytical Black-Scholes price
    bs_price = black_scholes_call(S0, K, T, r, sigma)

    # Monte Carlo simulation with all variance reduction techniques
    (mc_price_all_vr, duration_all_vr) = european_call_mc_sobol_vr(
        S0, K, T, r, sigma, n_paths,
        antithetic=True,
        control_variate=True,
        stratified=True
    )

    # Monte Carlo simulation without the new technique (for comparison)
    '''(mc_price_no_stratified, duration_no_stratified) = european_call_mc_sobol_vr(
        S0, K, T, r, sigma, n_paths,
        antithetic=True,
        control_variate=True,
        stratified=False
    )'''

    # Results
    print("\n--- European Call Option Pricing with Variance Reduction ---")
    print(f"Black-Scholes (Analytical):    {bs_price:.8f}")

    print("\n--- With Antithetic, Control Variate, and Stratified Sampling ---")
    abs_error_all_vr = abs(mc_price_all_vr - bs_price)
    rel_error_all_vr = abs_error_all_vr / bs_price * 100
    total_paths_all_vr = n_paths * 2  # because of antithetic
    print(f"MC Estimate:                   {mc_price_all_vr:.8f}")
    print(f"Absolute Error:                {abs_error_all_vr:.8e}")
    print(f"Relative Error:                {rel_error_all_vr:.8e}%")
    print(f"Paths Used:                    {total_paths_all_vr:,}")
    print(f"Time Elapsed:                  {duration_all_vr:.6f} seconds")

   
    print("\n--- With Antithetic and Control Variate (No Stratified Sampling) ---")
    '''
    abs_error_no_stratified = abs(mc_price_no_stratified - bs_price)
    rel_error_no_stratified = abs_error_no_stratified / bs_price * 100
    total_paths_no_stratified = n_paths * 2  # because of antithetic
    print(f"MC Estimate:                   {mc_price_no_stratified:.8f}")
    print(f"Absolute Error:                {abs_error_no_stratified:.8e}")
    print(f"Relative Error:                {rel_error_no_stratified:.8e}%")
    print(f"Paths Used:                    {total_paths_no_stratified:,}")
    print(f"Time Elapsed:                  {duration_no_stratified:.6f} seconds")'''


--- European Call Option Pricing with Variance Reduction ---
Black-Scholes (Analytical):    10.45058357

--- With Antithetic, Control Variate, and Stratified Sampling ---
MC Estimate:                   10.45058357
Absolute Error:                6.96437574e-09
Relative Error:                6.66410224e-08%
Paths Used:                    268,435,456
Time Elapsed:                  25.723573 seconds

--- With Antithetic and Control Variate (No Stratified Sampling) ---


In [5]:
import numpy as np
from scipy.stats import norm, qmc
import time

def black_scholes_call(S0, K, T, r, sigma, epsilon=1e-8):
    """ Black-Scholes formula for European call option price, with epsilon adjustment to avoid log(0). """
    S = np.maximum(S0, epsilon)  # Ensure S is not zero
    K = np.maximum(K, epsilon)  # Ensure K is not zero
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def delta(S, K, T, r, sigma, epsilon=1e-8):
    """Delta of a European call option with epsilon adjustment to avoid log(0)."""
    S = np.maximum(S, epsilon)
    K = np.maximum(K, epsilon)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

def gamma(S, K, T, r, sigma, epsilon=1e-8):
    """Gamma of a European call option with epsilon adjustment to avoid log(0)."""
    S = np.maximum(S, epsilon)
    K = np.maximum(K, epsilon)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

def timed(func):
    """A decorator that times the execution of a function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper

@timed
def european_call_mc_all_vr(S0, K, T, r, sigma, n_paths, antithetic=True, control_variate=True, stratified=True, 
                            moment_matching=False):
    """Monte Carlo simulation for European call option pricing using all variance reduction techniques with delta and gamma hedging."""
    
    # Generate Sobol sequences
    sampler = qmc.Sobol(d=1, scramble=True)
    sobol_samples = sampler.random(n_paths)
    
    # Stratified sampling: Ensure uniformity of z-values if enabled
    if stratified:
        sobol_samples = (np.arange(n_paths).reshape(-1, 1) + sobol_samples) / n_paths

    # Generate standard normal random variables
    z = norm.ppf(sobol_samples.flatten())
    
    # Moment matching: Standardize z-values if enabled
    if moment_matching:
        z = (z - np.mean(z)) / np.std(z, ddof=1)
    
    # Antithetic variates: Concatenate z and -z if enabled
    if antithetic:
        z = np.concatenate([z, -z])
    
    # Simulate the underlying asset's price at maturity (using geometric Brownian motion)
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    # Calculate the initial price and option payoff (European Call)
    call_payoff = np.maximum(ST - K, 0)
    discounted_call = np.exp(-r * T) * call_payoff

    # Hedging: Implement delta and gamma hedging in the simulation
    hedged_payoffs = []
    for i in range(0, len(ST), 2):  # Loop through each pair (since antithetic doubles the number of paths)
        # Initial stock price and volatility (from Sobol samples)
        S = S0 + (z[i] * 20)  # stock price from Sobol
        vol = 0.1 + (z[i + 1] * 0.2)  # volatility from Sobol
        
        # Delta and Gamma at time 0 (before any movement in stock price)
        delta_0 = delta(S, K, T, r, vol)
        gamma_0 = gamma(S, K, T, r, vol)

        # Simulate the stock price path using geometric Brownian motion
        path = np.zeros(100)
        path[0] = S
        for t in range(1, 100):
            z_t = np.random.normal(0, 1)  # Standard normal random variable
            path[t] = path[t-1] * np.exp((r - 0.5 * vol**2) * (T / 100) + vol * np.sqrt(T / 100) * z_t)

        # Hedging (adjusting portfolio based on delta and gamma)
        option_price = black_scholes_call(path[-1], K, T, r, vol)
        hedged_payoff = option_price - delta_0 * (path[-1] - S) - gamma_0 * (path[-1] - S)**2
        hedged_payoffs.append(hedged_payoff)

    # Control variate: The expected value of ST under the risk-neutral measure
    if control_variate:
        control_payoff = ST
        control_expected = S0 * np.exp(r * T)  # Black-Scholes expected value of ST
        # Compute covariance and variance of control variate
        cov = np.cov(discounted_call, control_payoff)[0, 1]
        var_control = np.var(control_payoff, ddof=1)
        # Optimal coefficient beta for control variate
        beta = cov / var_control
        # Adjust the payoff using control variate
        adjusted_payoff = discounted_call - beta * (control_payoff - control_expected)
        option_price = np.mean(adjusted_payoff)
    else:
        option_price = np.mean(discounted_call)

    return option_price

# Parameters
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1.0   # Time to maturity
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
n_paths = 2**12  # Number of Monte Carlo paths

# Analytical price (Black-Scholes)
bs_price = black_scholes_call(S0, K, T, r, sigma)

# Monte Carlo simulation with all variance reduction techniques
(mc_price_all_vr, duration_all_vr) = european_call_mc_all_vr(S0, K, T, r, sigma, n_paths)

# Results
print("\n--- European Call Option Pricing via Monte Carlo with All Variance Reduction Techniques and Hedging ---")
print(f"Black-Scholes (Analytical):    {bs_price:.8f}")
print(f"MC Estimate:                   {mc_price_all_vr:.8f}")
abs_error = abs(mc_price_all_vr - bs_price)
rel_error = abs_error / bs_price * 100
print(f"Absolute Error:                {abs_error:.8e}")
print(f"Relative Error:                {rel_error:.8e}%")
print(f"Paths Used:                    {n_paths * 2:,}")  # Because of antithetic
print(f"Time Elapsed:                  {duration_all_vr:.6f} seconds")



--- European Call Option Pricing via Monte Carlo with All Variance Reduction Techniques and Hedging ---
Black-Scholes (Analytical):    10.45058357
MC Estimate:                   10.45023028
Absolute Error:                3.53287479e-04
Relative Error:                3.38055264e-03%
Paths Used:                    8,192
Time Elapsed:                  3.003267 seconds


In [6]:
import numpy as np
from scipy.stats import norm, qmc
import time

def black_scholes_call(S0, K, T, r, sigma):
    """Vectorized Black-Scholes formula for European call option price."""
    S = np.maximum(S0, 1e-8)  # Ensure S is not zero
    K = np.maximum(K, 1e-8)   # Ensure K is not zero
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def timed(func):
    """A decorator that times the execution of a function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper

@timed
def european_call_mc_all_vr(S0, K, T, r, sigma, n_paths, antithetic=True, control_variate=True, stratified=True, moment_matching=False, epsilon=1e-5):
    """Monte Carlo simulation for European call option pricing using all variance reduction techniques with delta and gamma hedging (vectorized)."""
    
    # Generate Sobol sequences
    sampler = qmc.Sobol(d=1, scramble=True)
    sobol_samples = sampler.random(n_paths)
    
    # Stratified sampling: Ensure uniformity of z-values if enabled
    if stratified:
        sobol_samples = (np.arange(n_paths).reshape(-1, 1) + sobol_samples) / n_paths

    # Generate standard normal random variables (vectorized)
    z = norm.ppf(sobol_samples.flatten())
    
    # Moment matching: Standardize z-values if enabled
    if moment_matching:
        z = (z - np.mean(z)) / np.std(z, ddof=1)
    
    # Antithetic variates: Concatenate z and -z if enabled (vectorized)
    if antithetic:
        z = np.concatenate([z, -z])

    # Simulate the underlying asset's price at maturity (using geometric Brownian motion, vectorized)
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    # Option payoff (European Call, vectorized)
    call_payoff = np.maximum(ST - K, 0)

    # Discounted payoff (vectorized)
    discounted_call = np.exp(-r * T) * call_payoff

    # Hedging: Implement delta and gamma hedging (vectorized)
    S = S0 + (z * 20)  # stock prices from Sobol, vectorized
    vol = 0.1 + (z * 0.2)  # volatility from Sobol, vectorized
    
    # Estimate delta and gamma using finite differences (vectorized)
    # Perturb the stock price for delta and gamma estimates
    S_plus = S + epsilon  # Perturbed stock price for delta and gamma
    S_minus = S - epsilon
    
    # Estimate delta (finite difference)
    delta_0 = (black_scholes_call(S_plus, K, T, r, vol) - black_scholes_call(S_minus, K, T, r, vol)) / (2 * epsilon)
    
    # Estimate gamma (finite difference)
    gamma_0 = (black_scholes_call(S_plus, K, T, r, vol) - 2 * black_scholes_call(S, K, T, r, vol) + black_scholes_call(S_minus, K, T, r, vol)) / (epsilon**2)

    # Hedging payoffs (vectorized) using estimated delta and gamma
    hedged_payoffs = discounted_call - delta_0 * (ST - S) - gamma_0 * (ST - S)**2
    
    # Control variate: The expected value of ST under the risk-neutral measure
    if control_variate:
        control_payoff = ST
        control_expected = S0 * np.exp(r * T)  # Black-Scholes expected value of ST
        # Compute covariance and variance of control variate
        cov = np.cov(discounted_call, control_payoff)[0, 1]
        var_control = np.var(control_payoff, ddof=1)
        # Optimal coefficient beta for control variate
        beta = cov / var_control
        # Adjust the payoff using control variate
        adjusted_payoff = discounted_call - beta * (control_payoff - control_expected)
        option_price = np.mean(adjusted_payoff)
    else:
        option_price = np.mean(discounted_call)

    return option_price

# Parameters
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1.0   # Time to maturity
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
n_paths = 2**21  # Number of Monte Carlo paths

# Analytical price (Black-Scholes)
bs_price = black_scholes_call(S0, K, T, r, sigma)

# Monte Carlo simulation with all variance reduction techniques
(mc_price_all_vr, duration_all_vr) = european_call_mc_all_vr(S0, K, T, r, sigma, n_paths)

# Results
print("\n--- European Call Option Pricing via Monte Carlo with All Variance Reduction Techniques and Hedging (Vectorized) ---")
print(f"Black-Scholes (Analytical):    {bs_price:.8f}")
print(f"MC Estimate:                   {mc_price_all_vr:.8f}")
abs_error = abs(mc_price_all_vr - bs_price)
rel_error = abs_error / bs_price * 100
print(f"Absolute Error:                {abs_error:.8e}")
print(f"Relative Error:                {rel_error:.8e}%")
print(f"Paths Used:                    {n_paths * 2:,}")  # Because of antithetic
print(f"Time Elapsed:                  {duration_all_vr:.6f} seconds")



--- European Call Option Pricing via Monte Carlo with All Variance Reduction Techniques and Hedging (Vectorized) ---
Black-Scholes (Analytical):    10.45058357
MC Estimate:                   10.45058308
Absolute Error:                4.94909733e-07
Relative Error:                4.73571384e-06%
Paths Used:                    4,194,304
Time Elapsed:                  3.504482 seconds


In [7]:
import numpy as np
from scipy.stats import norm, qmc
import time


def timed(func):
    """A decorator that times the execution of a function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        return result, duration
    return wrapper


def black_scholes_call(S0, K, T, r, sigma):
    """Calculates the price of a European call option using the Black-Scholes formula.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock price.

    Returns:
        float: The Black-Scholes price of the call option.
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


@timed
def european_call_mc_sobol_vr(S0, K, T, r, sigma, n_paths, antithetic=True, control_variate=True, stratified=False):
    """Estimates the price of a European call option using Monte Carlo simulation with Sobol sequence and variance reduction techniques.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        T (float): Time to maturity (in years).
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock price.
        n_paths (int): Number of Sobol paths to generate (before considering antithetic variates).
        antithetic (bool): Whether to use antithetic variates to reduce variance.
        control_variate (bool): Whether to use the stock price at maturity and a digital call as control variates.
        stratified (bool): Whether to use stratified sampling on the Sobol sequence.

    Returns:
        tuple: A tuple containing the estimated option price and the execution duration.
    """
    if not isinstance(n_paths, int) or n_paths <= 0:
        raise ValueError("n_paths must be a positive integer")
    if T <= 0:
        raise ValueError("Time to maturity must be positive")

    # Generate Sobol sequence
    sampler = qmc.Sobol(d=1, scramble=True)
    sobol_samples = sampler.random(n=n_paths)

    if stratified:
        # Apply stratified sampling
        stratified_samples = (np.arange(n_paths).reshape(-1, 1) + sobol_samples) / n_paths
        z = norm.ppf(stratified_samples.flatten())
    else:
        z = norm.ppf(sobol_samples.flatten())

    if antithetic:
        z = np.concatenate([z, -z])
        n_total_paths = 2 * n_paths
    else:
        n_total_paths = n_paths

    # Simulate stock prices at maturity
    drift = (r - 0.5 * sigma**2) * T
    diffusion = sigma * np.sqrt(T) * z
    ST = S0 * np.exp(drift + diffusion)

    # Calculate discounted call payoffs
    discount_factor = np.exp(-r * T)
    call_payoff = np.maximum(ST - K, 0)
    discounted_call = discount_factor * call_payoff

    if control_variate:
        # Control variate 1: Stock price at maturity
        control_payoff_1 = ST
        control_expected_1 = S0 * np.exp(r * T)

        # Control variate 2: Discounted digital call payoff
        digital_call_payoff = np.where(ST > K, 1, 0)
        control_payoff_2 = np.exp(-r * T) * digital_call_payoff
        d1_bs = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2_bs = d1_bs - sigma * np.sqrt(T)
        control_expected_2 = np.exp(-r * T) * norm.cdf(d2_bs)

        # Calculate covariances and variances
        cov_dc_x1 = np.cov(discounted_call, control_payoff_1)[0, 1]
        cov_dc_x2 = np.cov(discounted_call, control_payoff_2)[0, 1]
        var_x1 = np.var(control_payoff_1, ddof=1)
        var_x2 = np.var(control_payoff_2, ddof=1)
        cov_x1_x2 = np.cov(control_payoff_1, control_payoff_2)[0, 1]

        # Calculate the denominator
        denominator = var_x1 * var_x2 - cov_x1_x2**2

        if denominator != 0:
            # Calculate beta coefficients
            beta1 = (cov_dc_x1 * var_x2 - cov_dc_x2 * cov_x1_x2) / denominator
            beta2 = (-cov_dc_x1 * cov_x1_x2 + cov_dc_x2 * var_x1) / denominator

            # Adjust the call price using the control variates
            adjusted_payoff = discounted_call - beta1 * (control_payoff_1 - control_expected_1) - beta2 * (control_payoff_2 - control_expected_2)
            option_price = np.mean(adjusted_payoff)
        else:
            option_price = np.mean(discounted_call) # Fallback if denominator is zero

    else:
        option_price = np.mean(discounted_call)

    return option_price


if __name__ == "__main__":
    # Parameters
    S0 = 100
    K = 100
    T = 1.0
    r = 0.05
    sigma = 0.2
    n_paths = 2**23# Increased significantly

    # Analytical Black-Scholes price
    bs_price = black_scholes_call(S0, K, T, r, sigma)

    # Monte Carlo simulation with all variance reduction techniques
    (mc_price_all_vr, duration_all_vr) = european_call_mc_sobol_vr(
        S0, K, T, r, sigma, n_paths,
        antithetic=True,
        control_variate=True,
        stratified=True
    )

    # Results
    print("\n--- European Call Option Pricing with Variance Reduction ---")
    print(f"Black-Scholes (Analytical):     {bs_price:.12f}")

    print("\n--- With Antithetic, Control Variate (Stock & Digital Call), and Stratified Sampling ---")
    abs_error_all_vr = abs(mc_price_all_vr - bs_price)
    rel_error_all_vr = abs_error_all_vr / bs_price * 100
    total_paths_all_vr = n_paths * 2  # because of antithetic
    print(f"MC Estimate:                     {mc_price_all_vr:.12f}")
    print(f"Absolute Error:                  {abs_error_all_vr:.2e}")
    print(f"Relative Error:                  {rel_error_all_vr:.2e}%")
    print(f"Paths Used:                      {total_paths_all_vr:,}")
    print(f"Time Elapsed:                    {duration_all_vr:.6f} seconds")


--- European Call Option Pricing with Variance Reduction ---
Black-Scholes (Analytical):     10.450583572186

--- With Antithetic, Control Variate (Stock & Digital Call), and Stratified Sampling ---
MC Estimate:                     10.450583139515
Absolute Error:                  4.33e-07
Relative Error:                  4.14e-06%
Paths Used:                      16,777,216
Time Elapsed:                    2.362419 seconds


In [40]:
import numpy as np
from numba import njit

@njit(fastmath=True, parallel=True)
def estPiStratifiedAnti2Improved(N):
    """
    Estimates Pi using Monte Carlo with stratified sampling and antithetic variates.
    This improved version ensures clarity and potentially better variance reduction.
    """
    U = np.random.rand(N)
    jVet = np.arange(1, N + 1)
    V1 = (U + jVet - 1) / N
    V2 = (jVet - U) / N

    # Estimating Pi using integral of 4 * sqrt(1 - x^2) from 0 to 1
    Y1 = 4 * np.sqrt(1 - V1**2)
    Y2 = 4 * np.sqrt(1 - V2**2)
    pi_est_sqrt_array = (Y1 + Y2) / 2.0
    pi_est_sqrt = np.mean(pi_est_sqrt_array)

    # Estimating Pi using integral of 4 / (1 + x^2) from 0 to 1
    X1 = 4 / (1 + V1**2)
    X2 = 4 / (1 + V2**2)
    pi_est_arctan_array = (X1 + X2) / 2.0
    pi_est_arctan = np.mean(pi_est_arctan_array)

    # Combining the two estimates
    combined_estimate = (pi_est_sqrt + pi_est_arctan) / 2.0

    # Standard error calculation based on the combined estimate for each stratum
    per_stratum_estimate = (np.sqrt(1 - V1**2) + np.sqrt(1 - V2**2) +
                            1 / (1 + V1**2) + 1 / (1 + V2**2))
    std_err = np.std(per_stratum_estimate) / np.sqrt(N)

    return combined_estimate, combined_estimate - std_err, combined_estimate + std_err

# Example usage:
if __name__ == '__main__':
    num_strata =  2*int(1e7)
    pi_estimate, lower_bound, upper_bound = estPiStratifiedAnti2Improved(num_strata)
    actual_pi = np.pi

    absolute_error = np.abs(pi_estimate - actual_pi)
    relative_error = absolute_error / actual_pi

    print(f"Estimated Pi:     {pi_estimate:.12f}")
    print(f"Actual Pi: 3.141592653589793") 
    print(f"Absolute Error:   {absolute_error:.12e}")
    print(f"Relative Error:   {relative_error:12e}")
    print(f"Confidence Interval (approx. 68%): [{lower_bound:.11}, {upper_bound:.11f}]")

Estimated Pi:     3.141592653590
Actual Pi:          
3.141592653589793
Absolute Error:   1.647570968544e-13
Relative Error:   5.244381e-14
Confidence Interval (approx. 68%): [3.1414233876, 3.14176191961]


In [ ]:
njit(fastmath=True, parallel=True)
def estPiStratifiedAnti2(N):
    U = np.random.rand(N)
    jVet = np.arange(1, N+1)
    V1 = (U+jVet-1)/N
    V2 = (jVet-U)/N
    X1 = 1.0/(1+(V1)**2)
    X2 = 1.0/(1+(V2)**2)
    Y1 = np.sqrt(1-(V1)**2) 
    Y2 = np.sqrt(1-(V2)**2)
    XY = X1+X2+Y1+Y2 
    res = np.mean(XY)
    return  res, res-np.std(XY)/np.sqrt(N), res+np.std(XY)/np.sqrt(N)
e7 = int(1e7)

pi_estimate = estPiStratifiedAnti2(2*e7)[0]

actual_pi = np.pi
absolute_error = np.abs(pi_estimate - actual_pi)
relative_error = absolute_error / actual_pi
print(f"Estimated Pi:     {pi_estimate:.12f}")
print(f"Absolute Error:   {absolute_error:.12e}")
print(f"Relative Error:   {relative_error:12e}")

In [82]:
import numpy as np
from numba import njit, prange
import time

@njit(fastmath=True, parallel=True)
def est_pi_stratified_antithetic(N):
    XY = np.empty(N, dtype=np.float64)
    for i in prange(N):
        u = np.random.rand()
        j = i + 1
        v1 = (u + j - 1) / N
        v2 = (j - u) / N
        x1 = 1.0 / (1.0 + v1 * v1)
        x2 = 1.0 / (1.0 + v2 * v2)
        y1 = np.sqrt(1.0 - v1 * v1)
        y2 = np.sqrt(1.0 - v2 * v2)
        XY[i] = x1 + x2 + y1 + y2

    res = np.mean(XY)
    std_err = np.std(XY) / np.sqrt(N)
    return res, res - std_err, res + std_err

# --- Execution ---
N = int(2e9)

start = time.time()
pi_estimate, ci_low, ci_high = est_pi_stratified_antithetic(N)
elapsed = time.time() - start

# --- Results ---
actual_pi = np.pi
absolute_error = abs(pi_estimate - actual_pi)
relative_error = absolute_error / actual_pi

print("\n--- Pi Estimation with Stratified + Antithetic (Numba Parallel) ---")
print(f"Samples used:      {N:,}")
print(f"Actual Pi:         3.141592653589793") 
print(f"Estimated Pi:      {pi_estimate:.16f}")
print(f"95% CI:            ({ci_low:.12f}, {ci_high:.12f})")
print(f"Absolute Error:    {absolute_error:.12e}")
print(f"Relative Error:    {relative_error:.12e}")
print(f"Time Elapsed:      {elapsed:.6f} seconds")



--- Pi Estimation with Stratified + Antithetic (Numba Parallel) ---
Samples used:      2,000,000,000
Actual Pi:         3.141592653589793
Estimated Pi:      3.1415926535897962
95% CI:            (3.141575726987, 3.141609580192)
Absolute Error:    3.108624468950e-15
Relative Error:    9.895059008998e-16
Time Elapsed:      7.589242 seconds


In [85]:
import numpy as np
from numba import njit, prange
from scipy.stats import qmc
import time

@njit(fastmath=True, parallel=True)
def est_pi_from_sobol(points):
    N = points.shape[0]  # Number of samples
    XY = np.empty(N, dtype=np.float64)
    for i in prange(N):
        u = points[i, 0]
        v = points[i, 1]
        x = 1.0 / (1.0 + u * u)
        y = np.sqrt(1.0 - v * v)
        antithetic_x = 1.0 / (1.0 + (1 - u) * (1 - u))
        antithetic_y = np.sqrt(1.0 - (1 - v) * (1 - v))
        XY[i] = x + y + antithetic_x + antithetic_y

    res = np.mean(XY)
    std_err = np.std(XY) / np.sqrt(N)
    return res, res - std_err, res + std_err

# --- Setup ---
N = 2**29# You’ll actually get 2x samples because of antithetic
sampler = qmc.Sobol(d=2, scramble=True)
sobol_points = sampler.random(N)  # Shape (N, 2)

# --- Execution ---
start = time.time()
pi_estimate, ci_low, ci_high = est_pi_from_sobol(sobol_points)
elapsed = time.time() - start

# --- Results ---
actual_pi = np.pi
absolute_error = abs(pi_estimate - actual_pi)
relative_error = absolute_error / actual_pi

print("\n--- Pi Estimation with Sobol + Antithetic + Numba ---")

print(f"Samples used:      {2 * N:,} (antithetic)")
print(f"Actual Pi:         3.141592653589793") 
print(f"Estimated Pi:      {pi_estimate:.16f}")
print(f"95% CI:            ({ci_low:.12f}, {ci_high:.12f})")
print(f"Absolute Error:    {absolute_error:.12e}")
print(f"Relative Error:    {relative_error:.12e}")
print(f"Time Elapsed:      {elapsed:.6f} seconds")



--- Pi Estimation with Sobol + Antithetic + Numba ---
Samples used:      1,073,741,824 (antithetic)
Actual Pi:         3.141592653589793
Estimated Pi:      3.1415926535897736
95% CI:            (3.141585397744, 3.141599909435)
Absolute Error:    1.953992523340e-14
Relative Error:    6.219751377084e-15
Time Elapsed:      3.600928 seconds


In [101]:
import numpy as np
from numba import njit, prange
import time

@njit(fastmath=True, parallel=True)
def est_pi_stratified_antithetic_optimized(N):
    sum_XY = 0.0
    sum_XY_sq = 0.0

    for i in prange(N):
        u = np.random.rand()
        j = i + 1
        j_div_N = j / N
        u_div_N = u / N

        v1 = j_div_N + u_div_N
        v2 = j_div_N - u_div_N

        # Manually clip the values of v1 and v2 to ensure they're within the range [-1, 1]
        v1 = max(-1.0, min(1.0, v1))
        v2 = max(-1.0, min(1.0, v2))

        # Computing x1, x2, y1, y2 safely
        x1 = 1.0 / (1.0 + v1 * v1)
        x2 = 1.0 / (1.0 + v2 * v2)

        # Safely calculate y1 and y2, ensuring no negative value inside sqrt
        y1 = np.sqrt(1.0 - v1 * v1)
        y2 = np.sqrt(1.0 - v2 * v2)

        val = x1 + x2 + y1 + y2
        sum_XY += val
        sum_XY_sq += val * val

    # Calculating mean and standard deviation after accumulating the sum
    mean = sum_XY / N
    std_dev = np.sqrt((sum_XY_sq / N) - mean ** 2)
    ci_low = mean - std_dev / np.sqrt(N)
    ci_high = mean + std_dev / np.sqrt(N)

    return mean, ci_low, ci_high

# --- Execution ---
N = int(4e9)  # Total samples (reduce to 2,000 for quick test)

start = time.time()
pi_estimate, ci_low, ci_high = est_pi_stratified_antithetic_optimized(N)
elapsed = time.time() - start

# --- Results ---
actual_pi = np.pi
absolute_error = abs(pi_estimate - actual_pi)
relative_error = absolute_error / actual_pi

print("\n--- Pi Estimation with Optimized Stratified + Antithetic (Numba Parallel) ---")
print(f"Samples used:      {N:,}")
print(f"Actual Pi:         3.141592653589793")
print(f"Estimated Pi:      {pi_estimate:.16f}")
print(f"95% CI:            ({ci_low:.12f}, {ci_high:.12f})")
print(f"Absolute Error:    {absolute_error:.12e}")
print(f"Relative Error:    {relative_error:.12e}")
print(f"Time Elapsed:      {elapsed:.6f} seconds")



--- Pi Estimation with Optimized Stratified + Antithetic (Numba Parallel) ---
Samples used:      4,000,000,000
Actual Pi:         3.141592653589793
Estimated Pi:      3.1415926532147900
95% CI:            (3.141580684300, 3.141604622130)
Absolute Error:    3.750031396521e-10
Relative Error:    1.193672067012e-10
Time Elapsed:      10.701652 seconds
